In [1]:
import glob
import pandas as pd
import numpy as np
import xarray as xr

# make a climatology from the CMIP6 model archive files

redundant with script CMIP6extract.csh

In [2]:
def fixmonth(dfile):

    """Fix CESM months since by default the timestamp is for the first day of
    the next month

    Parameters
    ----------

    dfile : xarray dataset
            Dataset containing time to fix

    Returns
    -------

    dfile : xarray dataset
            Fixed dataset
    """

    mytime = dfile['time'][:].data
    for time in range(mytime.size):
        if mytime[time].month > 1:
            mytime[time] = mytime[time].replace(month=mytime[time].month-1)
        elif mytime[time].month == 1:
            mytime[time] = mytime[time].replace(month=12)
            mytime[time] = mytime[time].replace(year=mytime[time].year-1)

    dfile = dfile.assign_coords(time=mytime)

    return dfile

In [3]:
# FIRST PLACE YOU NEED TO EDIT
classoutputdata = '/glade/derecho/scratch/bitz/ATMS559HW3classruns/' # stash the results here CHANGE NAME TO YOURS

case = 'b.e21.B1850.f19_g17.CMIP6-piControl-2deg.001'     # for the piControl for example  
#case = 'b.e21.BCO2x4.f19_g17.CMIP6-abrupt4xCO2-2deg.001' # or the 4XCO2

avar = 'VT' # let's say you want a climatology this variable 

In [4]:
casedir = '/glade/campaign/collections/cmip/CMIP6/timeseries-cmip6/'+case+'/atm/proc/tseries/month_1/'
files=sorted(glob.glob(casedir+case+'.cam.h0.'+avar+'.*.nc'))  
files   # this should provide you with a list of all the years possible, next you need to choose one file and start/end dates

['/glade/campaign/collections/cmip/CMIP6/timeseries-cmip6/b.e21.B1850.f19_g17.CMIP6-piControl-2deg.001/atm/proc/tseries/month_1/b.e21.B1850.f19_g17.CMIP6-piControl-2deg.001.cam.h0.VT.000101-005012.nc',
 '/glade/campaign/collections/cmip/CMIP6/timeseries-cmip6/b.e21.B1850.f19_g17.CMIP6-piControl-2deg.001/atm/proc/tseries/month_1/b.e21.B1850.f19_g17.CMIP6-piControl-2deg.001.cam.h0.VT.005101-010012.nc',
 '/glade/campaign/collections/cmip/CMIP6/timeseries-cmip6/b.e21.B1850.f19_g17.CMIP6-piControl-2deg.001/atm/proc/tseries/month_1/b.e21.B1850.f19_g17.CMIP6-piControl-2deg.001.cam.h0.VT.010101-015012.nc',
 '/glade/campaign/collections/cmip/CMIP6/timeseries-cmip6/b.e21.B1850.f19_g17.CMIP6-piControl-2deg.001/atm/proc/tseries/month_1/b.e21.B1850.f19_g17.CMIP6-piControl-2deg.001.cam.h0.VT.015101-020012.nc',
 '/glade/campaign/collections/cmip/CMIP6/timeseries-cmip6/b.e21.B1850.f19_g17.CMIP6-piControl-2deg.001/atm/proc/tseries/month_1/b.e21.B1850.f19_g17.CMIP6-piControl-2deg.001.cam.h0.VT.020101-02

In [5]:
# SECOND PLACE YOU MAY NEED TO EDIT
ncfile=files[0] # take the first of the files, here for example, you can choose any one
datestart = '001001'  # yyyymm
dateend   = '002912'

In [6]:
# we want these info variables in files of 3D variables
keepers=['P0',
 'hyai',
 'hyam',
 'hybi',
 'hybm']

ds=xr.open_dataset(ncfile)
ds=fixmonth(ds)

if ('lev' in ds[avar].dims):  # keep the keepers and add PS to the climatology, which is handy for regridding sigma to pressure
    climatology = ds[avar].groupby("time.month").mean("time")
    ds=ds[keepers]  # begin new Dataset with just the keepers
    PSncfile = ncfile.replace('.'+avar+'.', '.PS.')
    dsp=xr.open_dataset(PSncfile)
    dsp=fixmonth(dsp)
    PSpclimatology = dsp['PS'].groupby("time.month").mean("time")
    dsp = []
    ds['PS'] = PSpclimatology # now add PS climotology
    ds[avar] = climatology   # and the 3D climatology
    ds=ds.rename({'month':'time'})
    ds.to_netcdf(classoutputdata+case+'_'+avar+'_climo.nc')
else:                                                              # this is just a simple 2D field
    climatology = ds[avar].groupby("time.month").mean("time")
    climatology=climatology.rename({'month':'time'})
    climatology.to_netcdf(classoutputdata+case+'_'+avar+'_climo.nc')

In [7]:
ds = []

In [8]:
climatology = []